<a href="https://colab.research.google.com/github/stlyash/Cpp-STL-Examples/blob/main/nerpython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install spacy_transformers

!pip install -U spacy

In [3]:
import spacy
import spacy_transformers
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [4]:
!git clone https://github.com/stlyash/spacyResumeParcer.git

Cloning into 'spacyResumeParcer'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 65 (delta 1), reused 0 (delta 0), pack-reused 59
Unpacking objects: 100% (65/65), 7.81 MiB | 11.76 MiB/s, done.


base config file is downloaded from https://spacy.io/usage/training/#quickstart

In [5]:
cv_data = json.load(open('/content/spacyResumeParcer/data/training/train_data.json'))

In [6]:
len(cv_data)

200

In [7]:
!python -m spacy init fill-config /content/spacyResumeParcer/data/training/base_config.cfg /content/spacyResumeParcer/data/training/config.cfg

2023-04-06 12:53:19.154846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/spacyResumeParcer/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
from numpy import e
def get_spacy_doc (file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm (data):
    doc = nlp.make_doc(text) 
    annot =  annot['entities']
    ents = []
    entity_indices = []
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start,end))

      try:
        span = doc.char_span(start, end, label = label, alignment_mode = 'strict')
      except:
        continue
      
      if span is None:
        err_data = str([start,end]) + "    " + str(text) + "\n"
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass
      
  return db

In [9]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(cv_data,test_size = 0.3)
len(train),len(test)

(140, 60)

In [10]:
file  = open('error.txt','w')
db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')
file.close()

100%|██████████| 60/60 [00:00<00:00, 102.47it/s]


In [11]:
!python -m spacy train /content/spacyResumeParcer/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-04-06 12:53:37.079910: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-04-06 12:53:47,623] [INFO] Set up nlp object from config
[2023-04-06 12:53:47,634] [INFO] Pipeline: ['transformer', 'ner']
[2023-04-06 12:53:47,637] [INFO] Created vocabulary
[2023-04-06 12:53:47,638] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrain

## Testing Model

In [12]:
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
nlp = spacy.load('/content/output/model-best')

In [14]:
!pip install docx2txt
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3977 sha256=0e26699d6e3490b0f44084d96c6ed8cf483829569840121c9359092dcdddaa95
  Stored in directory: /root/.cache/pip/wheels/40/75/01/e6c444034338bde9c7947d3467807f889123465c2371e77418
Successfully built docx2txt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 6.9 MB/s eta 0:00:00


In [15]:
import docx2txt     # to convert docx files into text
import PyPDF2       # to convert pdf files into text

## Testing a single Resume

In [16]:

text = ''
fileName = '/content/spacyResumeParcer/inputFolder/1.pdf'
try:
    resume = docx2txt.process(fileName)
    for i in resume:
            if i == '\n':
                text += ' '
            else:
                text+=i
except:
    file = open(fileName,'rb')
    reader=PyPDF2.PdfReader(file)
    for pg in range(len(reader.pages)):
        page = reader.pages[pg]
        tex = page.extract_text()
        for i in tex:
            if type(i)==str and i == '\n':
                text += ' '
            elif type(i)==str:
                text+=i
    file.close()
text

'Personal  InformationCURRICULUM VITAE FullNames: Mike KisasatiWanaswa IDCardNo.22859930 PostalAddress: P.O. Box 85575 80100 Mombasa TelephoneNo.0717 550926 EmailAddress:mikewanaswa@gmail.co mLanguages : Well spoken English and  SwahiliPurpose To put in use the latest inventions in Telecommunication and Information Technology for a  positive impact in Individuals, Business Enterprises and Corporate Organizations. Work Experience Date :April 2011 – To Date Position :Fixed Data Network Technician Employer :Ben’s Electronics Services Ltd, Mombasa Duties : Survey, Installation, Integration, Maintenance, Support and  Decommissioning of Fixed Data Services using various Access Technologies (WIMAX, FIBER, MICROWAVES and Wi-Fi) for  SafaricomLtd. : Survey, Installation and Support of Ceragon’s IP20 Access Technology for  Airtel (K) : Survey , Installation and Support of Cambridge P2MP Solutions for  Safaricom. : Fiber Optics Splicing and terminations, deployment, support and  maintenance : WAN

In [17]:
# Displaying Texts and Labels of Entities
doc = nlp(text)
for ent in doc.ents:
  print(ent.text,"  :  ",ent.label_)

Mike KisasatiWanaswa   :   Name
Mombasa   :   Location
Cisco   :   Companies worked at
Inoorero University   :   College Name
Bungoma   :   Location
Bungoma   :   Location
Bungoma   :   Location


## Reading and Analyzing Bulk Resume(s)

In [18]:
# Keywords to search in the resume
keyString = "C++,C,Python,Java,Excel,MatLab"
keylist = keyString.split(',')
keylen = len(keylist)

In [ ]:
import os  
import pandas as pd
import re

pathinp = r'/content/spacyResumeParcer/inputFolder'
pathparent = r'..'
os.chdir(pathinp)

fileList = os.listdir()

fileScore = []
degree = []
fileLen = len(fileList)
nameList = []
foundKeywordList = []
emailList = []
files = 0
indicator = 0
for fileName in fileList:

    # Printing percentage of files processed
    files+=1
    if files%(fileLen//4) == 0:
        indicator+=25
        print("Processed {}%".format(indicator))

    # Reading text from a file into an array
    keyfound = 0
    text=''
    try:
        resume = docx2txt.process(fileName)
        for i in resume:
                if i == '\n':
                    text += ' '
                else:
                    text+=i
    except:
        file = open(fileName,'rb')
        reader=PyPDF2.PdfReader(file)
        for pg in range(len(reader.pages)):
            page = reader.pages[pg]
            tex = page.extract_text()
            for i in tex:
                if type(i)==str and i == '\n':
                    text += ' '
                elif type(i)==str:
                    text+=i
        file.close()
        
    textRaw = text
    textOriginal = text.split(' ')

    # Searching for email
    regex = r"([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+\.[a-zA-Z0-9_-]+) | ([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)"
    matches = re.search(regex, text)
    canEmail = ''
    try:
        canEmail = str((matches.group()).strip())
    except:
        canEmail = "<Not Found>"
    emailList.append(canEmail)

    # Searching Names
    doc = nlp(text)
    name = '<Not Found>'
    for ent in doc.ents:
      if str(ent.label_) == "Name":
        name = str(ent.text)
        break
    nameList.append(name)

    # Searching required keywords
    found_skills = ''
    for ent in doc.ents:
      if str(ent.label_) == "Skills":
        found_skills = str(ent.text)
    foundKeywords=''
    for ky in keylist:
        if found_skills.find(ky) != -1:
            keyfound += 1
            foundKeywords += ky +', '
    foundKeywordList.append(foundKeywords)
    fileScore.append(keyfound*100/keylen)

Token indices sequence length is longer than the specified maximum sequence length for this model (694 > 512). Running this sequence through the model will result in indexing errors


Processed 25%


In [ ]:
# Making a dataframe and exporting it to a csv
os.chdir(pathparent)
df = pd.DataFrame({"File Names" : fileList, r"% Keywords Matched" : fileScore,"Keywords Found":foundKeywordList,"Name":nameList,"Email":emailList})
df=df[df[r"% Keywords Matched"]!=0]
df = df.sort_values(r"% Keywords Matched",ascending=False)
print(df)
df.to_csv("/content/spacyResumeParcer/resumeResult.csv", index=False)
print("---  %s files scanned ---" % (fileLen))